$$ p_g\left(X_g\right)=\Phi\left(\frac{\gamma_g-w_g X_g}{\sqrt{1-w_g^2}}\right) $$

$$ p_g\left(X_g\right)=\Phi\left(aX_g+b\right) $$

$$ a = \frac{\gamma_g}{\sqrt{1-w_g^2}} $$

$$ b = -\frac{w_g}{\sqrt{1-w_g^2}} $$

The expressions of w and gamma as functions of a and b are:
$$ w_g = -\frac{b}{\sqrt{b^2 + 1}} $$
$$ \gamma_g = a \cdot \sqrt{1 - w_g^2} $$


In [4]:
import numpy as np

In [5]:
def w_calc_func(a, b):
    return - b/ np.sqrt(b**2 + 1)

def gamma_calc_func(a, b):
    return a * np.sqrt(1 - w_calc_func(a, b)**2)

def a_calc_func(w, gamma):
    return gamma / np.sqrt(1 - w**2)

def b_calc_func(w, gamma):
    return -w / np.sqrt(1 - w**2)

In [6]:
gamma_g = -1.65
w_g = 0.35

a = a_calc_func(w_g, gamma_g)
b = b_calc_func(w_g, gamma_g)
print(a, b)

-1.7614096918559583 -0.37363235887853663


In [7]:
w_calc_func(a, b), gamma_calc_func(a, b)

(0.35, -1.65)

In [15]:
from src.data_generator import generate_default_buckets
from src.ML_estimation import calculate_my_likelihood_arr
from scipy.stats import norm, binom
from src.sucess_probability import p_g
from scipy.optimize import minimize
from scipy.integrate import quad

In [16]:
def calculate_variable_changed_likelihood_arr(d_g_arr, n_g_arr, p_g, prob_dens_func, a, b):
    integrand = lambda x: np.prod(binom.pmf(d_g_arr, n_g_arr, norm.cdf(a*x+b))) * prob_dens_func(x)

    result, _ = quad(integrand, -3, 3)

    return result

In [17]:
# convert gamma_list [gamma_1, gamma_2, gamma_3] to [a_1, a_2, a_3]
gamma_g = [-2.9, -2.3, -1.6]
w_g = [0.45, 0.45, 0.45]
a = [a_calc_func(w_g[i], gamma_g[i]) for i in range(3)]
b = [b_calc_func(w_g[i], gamma_g[i]) for i in range(3)]
a, b

([-3.2473765635439547, -2.5755055503969295, -1.7916560350587338],
 [-0.5039032598602688, -0.5039032598602688, -0.5039032598602688])

In [37]:
# generate random number for seed between 1 and 1000
seed = 491
np.random.seed(seed)

# generate default buckets
factor_loading_list = [0.45, 0.45, 0.45]
num_of_obligors_list = [250, 250, 250]
gamma_list = [-2.9, -2.3, -1.6]
d_g_list = generate_default_buckets(factor_loading_list,num_of_obligors_list, gamma_list, 500)

# Different gamma and same factor loading parameter
d_g_arr = d_g_list
n_g_arr = np.array(num_of_obligors_list) * 500

# MLE condition and initial guess
initial_guess = np.array([-3.247, -2.576, -1.792, -0.50])
bounds = [(-5, 5), (-5, 5), (-5, 5), (-5, 5)]

# Function to be minimized in weight parameter
objective_function = lambda params: -np.log(calculate_variable_changed_likelihood_arr(d_g_arr, n_g_arr, p_g, norm.pdf, 
                                                                 np.repeat(params[3], 3), 
                                                                 params[0:3]))

result = minimize(objective_function,
                  initial_guess,
                  method="Nelder-Mead",
                  bounds=bounds,
                  options={
                      'disp': True})
# Method can be Nelder-Mead or Powell

# The optimal weight parameter
optimal_weight = result.x
print(f"The optimal weight parameter is {optimal_weight}")
print(result.message)

Optimization terminated successfully.
         Current function value: 18.658821
         Iterations: 61
         Function evaluations: 142
The optimal weight parameter is [-3.18606011 -2.51327555 -1.84717946 -0.51715289]
Optimization terminated successfully.


In [38]:
b_result = result.x[3]
a_result = [result.x[i] for i in range(3)]
w_result = [w_calc_func(a_result[i], b_result) for i in range(3)]
gamma_result = [gamma_calc_func(a_result[i], b_result) for i in range(3)]

w_result, gamma_result

([0.459360910748799, 0.459360910748799, 0.459360910748799],
 [-2.8300169821725167, -2.2324162911860133, -1.6407566283665866])

In [43]:
# Confidence interval for the ML parameter estimation
list_of_len_ts = [100, 200, 300, 500]
num_of_simulations = 10
factor_loading_list = [0.45, 0.45, 0.45]
pd_list = [0.0015, 0.0100, 0.0500]
gamma_list = norm.ppf(pd_list)
num_of_obligors_list = [400, 250, 100]

# Create a dictionary to store the results
# make the keys of the dictionary the length of the time series
results = {key: [] for key in list_of_len_ts}
for _ in range(num_of_simulations):
    for len_ts in list_of_len_ts:
        default_list = generate_default_buckets(factor_loading_list,num_of_obligors_list, gamma_list, time_points=len_ts)
        num_of_obligors_over_time = [x * len_ts for x in num_of_obligors_list]
        d_g_arr = np.array(default_list)
        n_g_arr = np.array(num_of_obligors_over_time)
        a_list = [a_calc_func(factor_loading_list[i], gamma_list[i]) for i in range(3)]
        b_list = [b_calc_func(factor_loading_list[i], gamma_list[i]) for i in range(3)]
        # Let a init the a_list rounded to 3 decimal places
        a_init = np.round(a_list, 3)
        b_init = np.round(b_list, 3)
        
        initial_guess = np.array(a_init)
        initial_guess = np.append(initial_guess, b_init[0])
        bounds = [(-5, 5), (-5, 5), (-5, 5), (-5, 5)]
        objective_function = lambda params: -np.log(calculate_variable_changed_likelihood_arr(d_g_arr, n_g_arr, p_g, norm.pdf, 
                                                                 np.repeat(params[3], 3), 
                                                                 params[0:3]))
        result = minimize(objective_function,
                  initial_guess,
                  method="Nelder-Mead",
                  bounds=bounds,
                  options={
                      'disp': False})
        
        b_result = result.x[3]
        a_result = [result.x[i] for i in range(3)]
        w_result = [w_calc_func(a_result[i], b_result) for i in range(3)]
        gamma_result = [gamma_calc_func(a_result[i], b_result) for i in range(3)]
        
        results[len_ts].append(w_result[0])

C:\Users\kissp\AppData\Local\Temp\ipykernel_17320\2100811897.py:4: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  result, _ = quad(integrand, -3, 3)


In [44]:
results

{100: [-3.954741676226937e-06,
  -4.169729787865443e-06,
  -2.2561185562170657e-05,
  8.472309165690303e-06,
  -3.878210393467119e-05,
  5.15501922976395e-06,
  7.147475223844656e-06,
  1.6333932857067317e-05,
  2.4238871395438084e-05,
  -2.430668682814424e-06],
 200: [0.2596942429758074,
  -1.9984866692475695e-05,
  -3.661882514874846e-05,
  1.4336196733970221e-05,
  -1.173912710743653e-05,
  0.46293277796209376,
  -4.1410029609961255e-06,
  -0.03478601988220862,
  0.45471911255550934,
  -0.018755188533190692],
 300: [0.44770750990613767,
  0.46105002822318286,
  0.038636172212812514,
  0.44878729199229284,
  0.4527353288824286,
  0.3778998767705431,
  0.4467530597305152,
  0.45317294122015234,
  0.45158992427371086,
  -7.511229378146125e-06],
 500: [0.4525865380784997,
  0.473848617511862,
  0.46350575090299184,
  -0.0030051004515318177,
  -4.774504982658473e-06,
  0.45142113909798204,
  0.45260120468632203,
  0.4377342192663827,
  0.44703355663224165,
  1.227431946423175e-05]}